# Wholesale customers Data Set (卸売業者の顧客データ)
https://archive.ics.uci.edu/ml/datasets/wholesale+customers <br>
ポルトガルの卸売業者の顧客のデータ（2011年(通年)、通貨単位の年間支出），この原データを入手して，ファイル名を変えて提供。

#### Warning回避
次のスクリプトは，下記のWariningを回避するためのものです。ただし，このWariningは無視しても良いので，次のスクリプトの実行は必須ではありません。

-  UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
- Ref.:https://stackoverflow.com/questions/69596239/how-to-avoid-memory-leak-when-dealing-with-kmeans-for-example-in-this-code-i-am

In [ ]:
import os
os.environ["OMP_NUM_THREADS"] = '2'

In [ ]:
# -*- coding: utf-8 -*-
import numpy as np
import pandas as pd

from sklearn.cluster import KMeans
from sklearn import datasets
#from sklearn.metrics import  accuracy_score, classification_report

import seaborn as sns

from matplotlib import cm
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
df_all = pd.read_csv('dataWholesale.csv')
df_all.head()

In [ ]:
#不要なカラムを削除
df1 = df_all.drop(['Channel', 'Region'], axis=1)
df1.head()

In [ ]:
# pandas.DataFrame -> numpy.ndarray
cstmr_data = df1.to_numpy() # customer data
cstmr_data

In [ ]:
print(type(cstmr_data))
print(cstmr_data.shape)
#cstmr_data[0:5] # numpy 配列の中身の確認

#### sklearn.cluster.KMeans
https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html
#### 注意
引数のrandom_state, n_initにより結果が異なる。
- random_state:クラスタ中心の選択をランダムとする。このため，クラスタ結果に影響を及ぼす。整数を与えれば確率変数の初期値は同じとなる。
- n_init: アルゴリズムが異なる重心から実行される回数。複数回の実行のうち最良の結果が出力される。'auto'の場合，他の引数のinitの値に依存する。引数initのデフォルトは8である。

In [ ]:
# kMeans
clstr = KMeans(n_clusters=4, n_init='auto', random_state=5).fit_predict(cstmr_data)

In [ ]:
# 結果（クラスタ番頭）の確認
for i in range(21):
    print('[',i,']:',clstr[i*20:i*20+20])

クラスタリングの結果をpandasのデータフレームに挿入する。

In [ ]:
df1['cluster_id'] = clstr
df1.head()

In [ ]:
# 各クラスタのデータ数の分布
df1['cluster_id'].value_counts()

In [ ]:
# クラスタ番号，それぞれにおける，商品の支出額の平均値
# range()の中の値を変えて，クラスタ番号に対応する項目を見る。
for cid in range(2,3):
    print('cluster_id=',cid)
#    print(df1[df1['cluster_id']==cid].mean())
    print(df1[df1['cluster_id']==cid])
    print()

In [ ]:
# クラスタ毎のグルーピングを行い，そのグループ内の平均値を求める
df2 = df1.groupby('cluster_id').mean()
#print(df2)
df2

In [ ]:
# グループ内の（不偏）標準偏差
df1.groupby('cluster_id').std(ddof=1)

pandasのplot.bar https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.plot.bar.html<br>
この引数の詳細　https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.plot.html<br>
Choosing Colormaps in Matplotlib https://matplotlib.org/stable/tutorials/colors/colormaps.html

In [ ]:
df2.plot.bar(alpha=0.6, figsize=(6,4), stacked=True, cmap='Set1')
#plt.savefig('fig_kMeans_WholeSale_Bar.pdf', bbox_inches='tight')
plt.show()

### 結果の考察：<br>
グラフから，クラスタ分類の特徴を推察する。
「相対的」とは，他のクラスタと比較することを意味する。また，数値は平均を表していることに留意。<br>
クラスタ番号 = 0 顧客 (276件)，Fresh以外の支出額が相対的にかなり低い<br>
クラスタ番号 = 1 顧客 ( 58件)，Freshの支出額が相対的に高い<br>
クラスタ番号 = 2 顧客 ( 11件)，Milk, Grocery, Detergents_Paperの支出額が相対的に大きい<br>
クラスタ番号 = 3 顧客 ( 95件)，FrechとGroceryの値が他のクラスタのものと異なる<br>

ただし，標準偏差が大きいので，平均値を表す棒グラフだけを見ての上記の考察が正しいか否かの検証は更に必要であろう。

In [ ]:
#pd.tools.plotting.scatter_matrix(df_1)
g = sns.pairplot(df1)

#plt.savefig('fig_kMeans_WholeSale_Pairplot.pdf', bbox_inches='tight')
plt.show()

上記の結果で，Grocery，Detergents_Paperの相関が比較的強いことが認められる。